## Define PCAP input and eve log output folders

Please change these varibles. If you use project docker container, then please create these on host .

In [ ]:
# search for pcap files here
PCAP_SRC = "/home/jovyan/work/data/pcap"
# per-pcap log output will be stored here
LOGS_DST = "/home/jovyan/work/data/logs"

Modify these if you wish. Should not be needed, unless you have custom compiled suricata or are reading PCAP files with different suffix.

In [ ]:
PATTERN = "*.pcap"
# Set global path to suricata install here
SURICATA = "suricata"
# Set global path to suricatasc unix socket connector utility here
SURICATASC = "suricatasc"
# Store suricata PID file here
PIDFILE = "/tmp/suricata-uxsock.pid"

## Python imports

In [ ]:
import os, signal
from glob import glob
import shutil
from subprocess import Popen, PIPE
import json

## Locate PCAP files

In [ ]:
FILES = glob(os.path.join(PCAP_SRC, PATTERN))
len(FILES)

## Set up Suricata daemon

Existing Suricata instance matching PID number in pidfile will be killed just to ensure that next command does not fail.

In [ ]:
if os.path.exists(PIDFILE):
    print("PID found, killing")
    with open(PIDFILE, "r") as handle:
        try:
            os.kill(int(handle.read()), signal.SIGKILL)
        except ProcessLookupError as err:
            print("pidfile found but no proc, just removing file")
    os.remove(PIDFILE)

Start suricata in unix-socket mode.

In [ ]:
p = Popen([SURICATA, 
           "--unix-socket",
           "--disable-detection",
           "--pidfile",
           PIDFILE,
           "-vvv",
           "-D"], stderr=PIPE, stdout=PIPE).communicate()

In [ ]:
print(p[0].decode("utf-8"))

In [ ]:
print(p[1].decode("utf-8"))

## Load PCAP files via unix socket

In [ ]:
if len(FILES) > 0:
    for fn in FILES:
        file = os.path.basename(fn)
        subdir = os.path.join(LOGS_DST, file)
        if os.path.exists(subdir):
            shutil.rmtree(subdir)
        os.mkdir(subdir)
        p = Popen([SURICATASC, 
                   "-c", 
                   "pcap-file {} {}".format(fn, subdir)], stderr=PIPE, stdout=PIPE).communicate()

## Check how many output files have been created

This should match the number of input files.

In [ ]:
FILES_EVE = glob(os.path.join(LOGS_DST + "/**/eve.json"))
len(FILES)

## Check status of PCAP parsing

Unix socket load only queues up PCAP files. Suricata will process them sequentially depending on resources available. Check this cell periodically to see how many are still queued.

In [ ]:
p = Popen([SURICATASC, "-c", "pcap-file-list"], stderr=PIPE, stdout=PIPE).communicate()

In [ ]:
status = json.loads(p[0].decode("utf-8"))
status["message"]["count"]